In [ ]:
import getpass
import os
import random
import sys
from datetime import datetime
from itertools import cycle
try:
    import numpy as np
    from PIL import Image
except ModuleNotFoundError:
    print('Enter the command: pip install numpy pillow')
    sys.exit()
def progress_bar(it, prefix='', size=60, out=sys.stdout):
        count = len(it)

        def show(j):
            x = int(size*j/count)
            print(f"{prefix}[{u'='*x}{('·'*(size-x))}]", end='\r', file=out, flush=True)
        show(0)
        for i, item in enumerate(it):
            yield item
            show(i+1)
        print('', flush=True, file=out)

class ImageCrypt():
    def __init__(self, password):
        self.password = password
    
    def encrypt(self, filename):
        self.encrypted_filename=""
        new_filename = self.new_filename('e', filename)
        if new_filename == 1:
            print(' encrypted filre already exists')
        else:
            try:
                start = datetime.now()
                im = Image.open(filename)
                encrypted = self.get_pixels(im)
                numbers = self.password_to_numbers(self.password)
               
                for i in progress_bar(numbers, 'Encryption: '):
                    encrypted = self.rail_fence_encrypt(encrypted, i)
                self.create_and_save_image(encrypted, im.width, im.height, new_filename)
                print(f'Encrypted image saved as {new_filename}')
                self.encrypted_filename=new_filename
                print('Time:', datetime.now() - start)
                return self.encrypted_filename
            except FileNotFoundError as error:
                print(error)
                
          
    def decrypt(self, filename, encrypted_filename):
            new_filename = self.new_filename('d', filename)
            if new_filename==1:
                print(' decrypted filre already exists')
                return 0
            ab='{}-e.png'.format(filename.split('.')[0])
            if   os.path.isfile(ab) or '-e' in filename:
                try:
                    attempts=2
                    while attempts>0:
                    # Prompt user for password during decryption
                        password = getpass.getpass(prompt='Enter decryption password again: ', stream=None)
                        if password != self.password:
                            attempts-=1
                            print("Error: Incorrect password for decryption.")
                            if attempts==0:
                                print("Maximum attempts reached")     
                                raise SystemExit
                        else:
                            break

                    start = datetime.now()
                    if '-e' in filename:
                        im = Image.open(filename)
                    else:
                        im=Image.open(ab)
                    decrypted = self.get_pixels(im)
                    numbers = self.password_to_numbers(self.password[::-1])
                    new_filename = self.new_filename('d', filename)
                    
                    for i in progress_bar(numbers, 'Decryption: '):
                        decrypted = self.rail_fence_decrypt(decrypted, i)
                    self.create_and_save_image(decrypted, im.width, im.height, new_filename)
                    print(f'Decrypted image saved as {new_filename}')
                    print('Time:', datetime.now() - start)
                    
                except FileNotFoundError as error:
                    print(error)
                    return
            else:
                print('No encrypted file found. The original image remains unchanged.')



    @staticmethod
    def get_pixels(im):
        print('Scanning...')
        colors = []
        for x in range(im.width):
            for y in range(im.height):
                color = im.getpixel((x, y))
                colors.append(color)
        return colors

    @staticmethod
    def password_to_numbers(password):
        numbers = [ord(i) for i in password]
        return numbers
    
    
    
    def rail_fence_encrypt(self, plaintext, rails):
        p = self.rail_pattern(rails)
        return sorted(plaintext, key=lambda i: next(p))

    def rail_fence_decrypt(self, ciphertext, rails):
        p = self.rail_pattern(rails)
        indexes = sorted(range(len(ciphertext)), key=lambda i: next(p))
        result = [''] * len(ciphertext)
        for i, c in zip(indexes, ciphertext):
            result[i] = c
        return result

    @staticmethod
    def rail_pattern(n):
        r = list(range(n))
        return cycle(r + r[-2:0:-1])
    @staticmethod
    def new_filename(mode, filename):
        base_name, extension = os.path.splitext(filename)

        if mode.lower() == 'e':
            mode_suffix = '-e'
        elif mode.lower() == 'd':
            mode_suffix = '-d'
        else:
            print("Invalid mode. Please enter 'E' for encryption or 'D' for decryption.")
            return None
        new_filename = f'{filename[:len(filename) - 4]}-{mode}.png'
        if os.path.exists(new_filename):
            return 1
        else:
            return new_filename

    @staticmethod
    def create_and_save_image(colors, width, height, filename):
        print('Creation...')
        im = Image.new('RGB', (width, height))
        image_array = np.array(im)
        i = 0
        for x in range(width):
            for y in range(height):
                r = colors[i][0]
                g = colors[i][1]
                b = colors[i][2]
                image_array[y][x] = (r, g, b)
                i += 1
        new_image = Image.fromarray(image_array, 'RGB')
        new_image.save(filename)


    def main():
       
        encrypted_filename=""
        while True:
            filename = input('Enter image filename or quit[Q/q]: ')
            if filename.lower() == 'q' or filename.lower()=='Q':
                print('Quitting...')
                break
            
            if filename.lower().endswith(('-e.png')):
                x=1
                while x==1:
                    password=getpass.getpass(prompt="Enter the correct password for decryption:",stream=None)
                    image_crypt=ImageCrypt(password)
                    if len(password) >= 8:
                        x = 0
                        image_crypt.decrypt(filename,encrypted_filename)
                    else:
                        print('Error: Minimum password length: 8')
                    
            
            elif filename.lower().endswith(('png', 'jpg')):
                m = 1
                while m == 1:
                    password = getpass.getpass(prompt='Enter password: ', stream=None)

                    # Create an instance of ImageCrypt with a placeholder password
                    image_crypt = ImageCrypt(password)
                    if len(password) >= 8:
                        m = 0
                    else:                
                        print('Error: Minimum password length: 8')
                        
            else:
                print('!!!!!Only .png/.jpg format or Q and q\nPlease enter a valid image filename.')

            n=1
            while n == 1:
                while True:
                    mode = input('Do you want to encrypt or decrypt the image or exit [E/D/X]? ')
                    if mode == 'd' or mode=='D':
                        if encrypted_filename !="":
                            image_crypt.decrypt(filename,encrypted_filename)
                        else:
                            image_crypt.decrypt(filename,encrypted_filename)
                        m = 0
                        n = 0
                        x=0
                    elif mode == 'e' or mode=='E':
                        encrypted_filename=image_crypt.encrypt(filename)
                        n = 0
                        m = 0
                        x=0
                    elif mode=='x' or mode=='X':
                        print('Exiting...')
                        break
                    else:
                        print("Enter correct option\n")
    

if __name__ == '__main__':
    ImageCrypt.main()
